In [16]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import pickle

In [17]:
df = pd.read_csv('hackathon.csv')

In [18]:
df.head()

,Unnamed: 0,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,2,1,TRANSFER,181.0,C1305486145,181.0,0.0,C553264065,0.0,0.0,1,0
1,3,1,CASH_OUT,181.0,C840083671,181.0,0.0,C38997010,21182.0,0.0,1,0
2,251,1,TRANSFER,2806.0,C1420196421,2806.0,0.0,C972765878,0.0,0.0,1,0
3,252,1,CASH_OUT,2806.0,C2101527076,2806.0,0.0,C1007251739,26202.0,0.0,1,0
4,680,1,TRANSFER,20128.0,C137533655,20128.0,0.0,C1848415041,0.0,0.0,1,0


In [19]:
df.reset_index(drop=True)

,Unnamed: 0,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
1,3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
2,251,1,TRANSFER,2806.00,C1420196421,2806.00,0.00,C972765878,0.00,0.00,1,0
3,252,1,CASH_OUT,2806.00,C2101527076,2806.00,0.00,C1007251739,26202.00,0.00,1,0
4,680,1,TRANSFER,20128.00,C137533655,20128.00,0.00,C1848415041,0.00,0.00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16412,2596607,207,PAYMENT,18588.97,C264877418,393507.58,374918.61,M1037637931,0.00,0.00,0,0
16413,1740940,161,TRANSFER,94957.41,C1966589833,0.00,0.00,C37728871,827932.92,922890.33,0,0
16414,1107941,130,PAYMENT,4646.91,C821923409,336051.82,331404.91,M589073853,0.00,0.00,0,0
16415,5107980,355,DEBIT,2804.53,C1742662400,895.00,0.00,C1661446958,248929.59,251734.12,0,0


In [20]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# data = df.copy()

# #Ivde extracting nameOrig and nameDest where C is present
# data['OrigC']=data['nameOrig'].apply(lambda x: 1 if str(x).find('C')==0 else 0)
# data['DestC']=data['nameDest'].apply(lambda x: 1 if str(x).find('C')==0 else 0)

# #Ivde creating new feature for transfer and cash_out
# data['TRANSFER']=data['type'].apply(lambda x: 1 if x=='TRANSFER' else 0)
# data['CASH_OUT']=data['type'].apply(lambda x: 1 if x=='CASH_OUT' else 0)

droplist=['step','isFlaggedFraud','type','nameDest','nameOrig']

#Preprocessing function 
def data_preprocessing(data):
    
    #Ivde extracting nameOrig and nameDest where C is present
    
    
#     data['OrigC']=data['nameOrig'].apply(lambda x: 1 if str(x).find('C')==0 else 0)
#     data['DestC']=data['nameDest'].apply(lambda x: 1 if str(x).find('C')==0 else 0)

    #Ivde creating new feature for transfer and cash_out
    data['TRANSFER']=data['type'].apply(lambda x: 1 if x=='TRANSFER' else 0)
    data['CASH_OUT']=data['type'].apply(lambda x: 1 if x=='CASH_OUT' else 0)
    
    #Ivde extracting error in account balances
    data['Amount Error']=(abs(data['oldbalanceOrg']-data['newbalanceOrig'])-data['amount'])
          
    data = data.drop(labels=droplist,axis=1)
    
    return data

In [22]:
def model_result(model,x_test,y_test):
    y_pred=model.predict(x_test)
    print('F1-score :',(f1_score(y_test,y_pred)))
    print('Confusion_matrix : ')
    print(confusion_matrix(y_test,y_pred))
    print("accuracy_score : ", end = '')
    print(accuracy_score(y_test,y_pred))
    print("classification_report")
    print(classification_report(y_test,y_pred))

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

In [24]:
df = data_preprocessing(df)

In [25]:
df['isFraud'].value_counts()

1    8213
0    8204
Name: isFraud, dtype: int64

In [26]:

X = df.drop('isFraud',axis=1)
Y = df['isFraud']
X = scaler.fit_transform(X) 

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3,random_state=42, shuffle=True)
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
model_result(rf_model,X_test,y_test)

F1-score : 0.9979773462783171
Confusion_matrix : 
[[2449    0]
 [  10 2467]]
accuracy_score : 0.9979699553390174
classification_report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2449
           1       1.00      1.00      1.00      2477

    accuracy                           1.00      4926
   macro avg       1.00      1.00      1.00      4926
weighted avg       1.00      1.00      1.00      4926



In [27]:
filename = 'final_model.sav'
pickle.dump(rf_model, open(filename, 'wb'))

In [28]:
X.columns

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [30]:
lst = [562736,
      287424,
      'C276',
     523875,
      134874,
      'C101739',
      92742,
      9029842,
      'TRANSFER']
#1	CASH_OUT	2806.00	C2101527076	2806.00	0.00	C1007251739	26202.00	0.00	1	

arr = np.array(lst).reshape(1,-1)

def data_preprocessing(data):
    
    #Ivde extracting nameOrig and nameDest where C is present
    
    droplist=['step','type','nameDest','nameOrig']

    
#     data['OrigC']=data['nameOrig'].apply(lambda x: 1 if str(x).find('C')==0 else 0)
#     data['DestC']=data['nameDest'].apply(lambda x: 1 if str(x).find('C')==0 else 0)

    #Ivde creating new feature for transfer and cash_out
    data['TRANSFER']=data['type'].apply(lambda x: 1 if x=='TRANSFER' else 0)
    data['CASH_OUT']=data['type'].apply(lambda x: 1 if x=='CASH_OUT' else 0)
    
    #Ivde extracting error in account balances
    data['Amount Error']=(abs(data['oldbalanceOrg']-data['newbalanceOrig'])-data['amount'])
       
    data = data.drop(labels=droplist,axis=1)
    
    return data



df = pd.DataFrame(data = arr,
                  columns = ['step', 'amount', 'nameOrig','oldbalanceOrg', 'newbalanceOrig', 'nameDest', 
                  'oldbalanceDest', 'newbalanceDest', 'type'])

df['step'] = df['step'].astype(int)
df['amount'] = df['amount'].astype(float)
df['oldbalanceOrg'] = df['oldbalanceOrg'].astype(float)
df['newbalanceOrig'] = df['newbalanceOrig'].astype(float)
df['oldbalanceDest'] = df['oldbalanceDest'].astype(float)
df['newbalanceDest'] = df['newbalanceDest'].astype(float)    


df = data_preprocessing(df)

model = pickle.load(open('final_model.sav','rb'))

prediction = model.predict(df)
if prediction[0]==1:
        print('fraud')
elif prediction[0] ==0:
        print('safe')

fraud


In [32]:
#X_train.columns

In [33]:
filename = 'scaler.sav'
pickle.dump(scaler, open(filename, 'wb'))

In [34]:
d = pd.read_csv('hackathon.csv')

In [35]:
len(d[(d['type']=='CASH_OUT') & d['isFraud']==1])

4116

In [36]:
len(d[(d['type']=='TRANSFER') & d['isFraud']==1])

4097

In [37]:
len(d[(d['type']=='CASH_OUT') & d['isFraud']==0])

12301

In [38]:
len(d[(d['type']=='TRANSFER') & d['isFraud']==0])

12320

In [39]:
d.columns

Index(['Unnamed: 0', 'step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg',
       'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest',
       'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [40]:
len(d[d['isFraud']==1])

8213

In [41]:
len(d[d['isFraud']==0])

8204

In [42]:
len(d)

16417

In [43]:
len(d[d['type']=='CASH_OUT'])

7042

In [44]:
len(d[d['type']=='TRANSFER'])

4791

In [45]:
d['type'].unique()

array(['TRANSFER', 'CASH_OUT', 'CASH_IN', 'PAYMENT', 'DEBIT'],
      dtype=object)